In [1]:
import os

os.chdir("../..")

os.getcwd()

'c:\\Users\\mquick\\Documents\\zeitgeist blog\\projects\\geopolitical_proximity'

In [2]:
from bq.bq_tables import boundaries, nodes
import geopandas as gpd
from google.cloud.bigquery import Client, Table
from settings import PROJECT
from shapely import wkt

In [3]:
location_boundaries = gpd.read_file("./notebooks/nodes/data/world-administrative-boundaries.shp")

# source: https://public.opendatasoft.com/explore/dataset/world-administrative-boundaries/export/

In [5]:
location_boundaries.head(3)

,iso3,status,color_code,name,continent,region,iso_3166_1_,french_shor,geometry
0,MNP,US Territory,USA,Northern Mariana Islands,Oceania,Micronesia,MP,Northern Mariana Islands,"MULTIPOLYGON (((145.63331 14.91236, 145.62412 ..."
1,None,Sovereignty unsettled,RUS,Kuril Islands,Asia,Eastern Asia,None,Kuril Islands,"MULTIPOLYGON (((146.68274 43.70777, 146.66664 ..."
2,FRA,Member State,FRA,France,Europe,Western Europe,FR,France,"MULTIPOLYGON (((9.4475 42.68305, 9.45014 42.63..."


In [4]:
bq = Client(PROJECT)

In [5]:
job = bq.query(f"SELECT DISTINCT id AS node_id, iso3 FROM `{nodes.id}`")
node_ids = job.result().to_dataframe()

c:\Users\mquick\Documents\zeitgeist blog\projects\geopolitical_proximity\venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [6]:
boundaries.schema

[SchemaField('node_id', 'INT64', 'REQUIRED', None, None, (), None),
 SchemaField('geometry', 'STRING', 'REQUIRED', None, None, (), None)]

In [7]:
node_boundaries = location_boundaries.merge(node_ids, on="iso3", how="right").loc[:, ["node_id", "geometry"]]

In [8]:
polys_valid = node_boundaries["geometry"].apply(lambda x: x.is_valid)

In [9]:
all(polys_valid)

True

In [10]:
node_boundaries[node_boundaries.isna().any(axis=1)]

,node_id,geometry


In [11]:
boundaries_table = Table(table_ref=boundaries.id, schema=boundaries.schema)

In [13]:
node_boundaries["geometry"] = node_boundaries["geometry"].apply(lambda x: x.wkt)

C:\Users\mquick\AppData\Local\Temp\ipykernel_57608\1815835886.py:1: UserWarning: Geometry column does not contain geometry.
  node_boundaries["geometry"] = node_boundaries["geometry"].apply(lambda x: x.wkt)


In [14]:
# bq.create_table(table=boundaries_table)

Table(TableReference(DatasetReference('zeitgeist-blog-staging', 'geopolitical_proximity'), 'boundaries'))

In [15]:
# bq.load_table_from_dataframe(dataframe=node_boundaries, destination=boundaries_table)

LoadJob<project=zeitgeist-blog-staging, location=US, id=13de9c5a-d7b8-4867-96ee-7f2342a68245>